# understat 사이트에서 2014년도 부터 2020 까지년도의 잉글랜드 프리미어 리그 선수 데이터 수집.

In [2]:
#필요한 라이브러리 
from selenium import webdriver
from pandas.io.html import read_html
import time
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

browser=webdriver.Chrome()
browser.maximize_window()
sleep_time=2

#delay 함수 제작
#너무 빠르게 지나가면 인식이 안될경우가있기 때문에 생성.
def delay():
    time.sleep(sleep_time)

#초기 테이블 설정.
url="https://understat.com/league/EPL/2021"
browser.get(url)
delay()
#원하는 데이터를 카테고리에 띄우기 위한 작업
browser.find_element_by_xpath('//*[@id="league-players"]/div[1]/button').click()
delay()
browser.find_element_by_xpath('//*[@id="league-players"]/div[2]/div[2]/div/div[12]/div[2]/label').click()
delay()
browser.find_element_by_xpath('//*[@id="league-players"]/div[2]/div[2]/div/div[13]/div[2]/label').click()
delay()
browser.find_element_by_xpath('//*[@id="league-players"]/div[2]/div[2]/div/div[17]/div[2]/label').click()
delay()
browser.find_element_by_xpath('//*[@id="league-players"]/div[2]/div[3]/a[2]').click()

#년도 별로 데이터 수집.
years=list(range(2014,2021))

for year in years:

    # 1) 페이지 바꾸기.
    url=f"https://understat.com/league/EPL/{year}"
    browser.get(url)
    delay()

    # 2) df 생성.
    df=pd.DataFrame(columns=['№','Player','Team', 'Apps', 'Min', 'G','A' ,'xG','xA','xGChain', 'xGBuildup' ,'xG90', 'xA90', 'xG90 + xA90'])
        
        ## 한 페이지에 선수 10명이며, 상위 300명의 데이터를 수집하겠습니다. 30페이지.
        ## 홈페이지를 보면 1~4번째 버튼까지 쓰다가 5번쨰 버튼 부터는 끝 숫자에서 2번쨰 전까지는 계속 5번쨰 숫자를 쓴다.
        ## 그러므로 1~4와 5~30을 구분해서 데이터 수집.
    
    delay()
    #상위 300명의 플레이러를 데이터 수집 (1페이지에 10명 이므로 30페이지)
    for i in range(1,5):
        browser.find_element_by_css_selector('#league-players > div.table-control-panel > ul > li:nth-child({})'.format(i)).click()
        delay()
        table=browser.find_element_by_xpath('//*[@id="league-players"]')
        table_html= table.get_attribute('innerHTML')
        df2=read_html(table_html)[0]
        df=pd.concat([df,df2],axis=0)
    for i in range(5,31):
        browser.find_element_by_css_selector('#league-players > div.table-control-panel > ul > li:nth-child(5)').click()
        delay()
        table=browser.find_element_by_xpath('//*[@id="league-players"]')
        table_html= table.get_attribute('innerHTML')
        df2=read_html(table_html)[0]
        df=pd.concat([df,df2],axis=0)
        
    browser.close()
    data=df

    # 3) 문제점 해결.
        ##문제점 :  페이지마다 테이블 마지막 row에 통계 데이터가 매번 들어가기떄문에 그부분 row를 삭제해야한다.
        ##해결책 :  통계 데이터에는 선수이름이 NaN이므로  NAN 값이 포함된 row들 삭제
    data=data.dropna()

        ##문제점 : 지저분한 인덱스
        ##해결책 : 인덱스 재 설정.
    data=data.reset_index()
    data.drop(['№','index'],axis=1,inplace=True)
    
        ##문제점 : 비슷한 의미의 데이터 삭제.(ex xG<->xG90 ,xA<->xA90)
        ##해결책 : 비슷한 의미의 컬럼 삭제.
    data.drop(['xG','xA'],axis=1,inplace=True)

    # 4) 정리된 data를 csv파일로 저장.
    data.to_csv(f'EPL_{year}-{year+1}_Player_data.csv',sep=',')

KeyboardInterrupt: 